In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7NNKmNQtyUlTXjM15ho-v34Bt-d7WR-PTU49-5AIYcSTRjwi0k6JI


In [ ]:
!yes | unzip /content/gdrive/MyDrive/151B_data/new_train.zip > /dev/null
!yes | unzip /content/gdrive/MyDrive/151B_data/new_val_in.zip > /dev/null

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import os, os.path 
import numpy 
import pickle
from glob import glob
from tqdm import tqdm


"""Change to the data folder"""
new_path = "/content/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [ ]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=new_path)

In [ ]:
batch_sz = 100
import numpy as np
import random

def my_collate_train(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    batch_inp = []
    batch_out = []

    for scene in batch:
      agent = scene['agent_id']
      target = 0
      for x in range(len(scene['track_id'])):
        if scene['track_id'][x][0] == agent:
          target = x
      inp = [scene['p_in'][target], scene['v_in'][target]]
      out = [scene['p_out'][target], scene['v_out'][target]]
      batch_inp.append(inp)
      batch_out.append(out)

    # scene level #####################
    # batch_inp = []
    # batch_out = []
    # for scene in batch:
    #   mask = scene['car_mask'].flatten()==1
    #   # print(np.count_nonzero(mask))
    #   inp = [scene['p_in'][mask], scene['v_in'][mask]]
    #   out = [scene['p_out'][mask], scene['v_out'][mask]]
    #   batch_inp.append(inp)
    #   batch_out.append(out)
    ####################################


    inp = torch.LongTensor(batch_inp)
    out = torch.LongTensor(batch_out)
    return [inp, out]

def my_collate_train_multiple(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    batch_inp = []
    batch_out = []

    for scene in batch:
      agent = scene['agent_id']
      target = 0
      for x in range(len(scene['track_id'])):
        if scene['track_id'][x][0] == agent:
          target = x
      inp = [scene['p_in'][target], scene['v_in'][target]]
      out = [scene['p_out'][target], scene['v_out'][target]]

      other_in = np.zeros((5,2,19,2)) # need to permute to 2,5,19,2 later
      other_out = np.zeros((5,2,30,2))

      other_in[0] = inp
      other_out[0] = out

      mask = scene['car_mask'].flatten()==1
      mask = np.delete(mask,target)
      mask = np.where(mask == True)[0]

      if(len(mask)>=4):
        temp = random.sample(mask.tolist(),4)
        for i in range(len(temp)):
          other_in[i+1] = [scene['p_in'][temp[i]], scene['v_in'][temp[i]]]
          other_out[i+1] = [scene['p_out'][temp[i]], scene['v_out'][temp[i]]]
      else:
        for i in range(len(mask)):
          other_in[i+1] = [scene['p_in'][mask[i]], scene['v_in'][mask[i]]]
          other_out[i+1] = [scene['p_out'][mask[i]], scene['v_out'][mask[i]]]
        for i in range(4-len(mask)):
          other_in[i+1+len(mask)] = inp
          other_out[i+1+len(mask)] = out

      batch_inp.append(other_in.tolist())
      batch_out.append(other_out.tolist())

    # scene level #####################
    # batch_inp = []
    # batch_out = []
    # for scene in batch:
    #   mask = scene['car_mask'].flatten()==1
    #   # print(np.count_nonzero(mask))
    #   inp = [scene['p_in'][mask], scene['v_in'][mask]]
    #   out = [scene['p_out'][mask], scene['v_out'][mask]]
    #   batch_inp.append(inp)
    #   batch_out.append(out)
    ####################################


    inp = torch.LongTensor(batch_inp)
    inp = inp.permute(2,0,1,3,4) # p/v, batch, cars, points, x/y
    out = torch.LongTensor(batch_out)
    out = out.permute(2,0,1,3,4) 
    return [inp, out]

def my_collate_val(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    inp = [[scene['p_in'], scene['v_in']] for scene in batch]
    mask = [scene['car_mask'] for scene in batch]

    inp = torch.LongTensor(inp)
    mask = torch.LongTensor(mask)
    return [inp, mask]



In [ ]:
# don't run

train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)

# max_p_x = 0
# min_p_x = 5000
# max_p_y = 0
# min_p_y = 5000
# max_v_x = 0
# min_v_x = 100
# max_v_y = 0
# min_v_y = 100

# for i_batch, sample_batch in enumerate(train_loader):
#     inp, out = sample_batch
#     for i in inp:
#       if i[0][:,0].max()> max_p_x:
#         max_p_x = i[0][:,0].max()
#       if i[0][:,0].min() < min_p_x:
#         min_p_x = i[0][:,0].min()
#       if i[0][:,1].max()> max_p_y:
#         max_p_y = i[0][:,1].max()
#       if i[0][:,1].min() < min_p_y:
#         min_p_y = i[0][:,1].min()

#       if i[1][:,0].max()> max_v_x:
#         max_v_x = i[1][:,0].max()
#       if i[1][:,0].min()< min_v_x:
#         min_v_x = i[1][:,0].min()
#       if i[1][:,1].max()> max_v_y:
#         max_v_y = i[1][:,1].max()
#       if i[1][:,1].min()< min_v_y:
#         min_v_y = i[1][:,1].min()

#     for o in out:
#       if o[0][:,0].max()> max_p_x:
#         max_p_x = o[0][:,0].max()
#       if o[0][:,0].min() < min_p_x:
#         min_p_x = o[0][:,0].min()
#       if o[0][:,1].max()> max_p_y:
#         max_p_y = o[0][:,1].max()
#       if o[0][:,1].min() < min_p_y:
#         min_p_y = o[0][:,1].min()

#       if o[1][:,0].max()> max_v_x:
#         max_v_x = o[1][:,0].max()
#       if o[1][:,0].min()< min_v_x:
#         min_v_x = o[1][:,0].min()
#       if o[1][:,1].max()> max_v_y:
#         max_v_y = o[1][:,1].max()
#       if o[1][:,1].min()< min_v_y:
#         min_v_y = o[1][:,1].min()

#     if(i_batch%100==0):
#       print(i_batch)

max = 0
min = 1000
factor = 0

for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
        
    for i in range(len(inp[0])):
      for c in range(len(inp[0][i])):
        disp = torch.unsqueeze(inp[0][i][c][0],0)
        x = torch.repeat_interleave(disp,19,0)
        displacement_in = torch.max(torch.abs(inp[0][i][c] - x))

        if(torch.min(inp[0][i][c] - x)<min):
          min = torch.min(inp[0][i][c] - x)

        if(torch.max(inp[0][i][c] - x)>max):
          max = torch.max(inp[0][i][c] - x)  

        x = torch.repeat_interleave(disp,30,0)
        displacement_out = torch.max(torch.abs(out[0][i][c] - x))

        if(torch.min(out[0][i][c] - x)<min):
          min = torch.min(out[0][i][c] - x)

        if(torch.max(out[0][i][c] - x)>max):
          max = torch.max(out[0][i][c] - x)  

        if(displacement_in!=0):
          temp = displacement_out/displacement_in
          if(temp>factor):
            factor = temp
    if(i_batch%100==0):
      print(i_batch, factor, max, min)
# print(max_p_x, min_p_x, max_p_y, min_p_y) # [4800,-50,4800,-50]
# print(max_v_x, min_v_x, max_v_y, min_v_y) # [200,-200,200,-200]

In [ ]:
'''
start with initial 19 points and zero fill remaining
'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2,64) # N, 19, 2
        self.v_in = nn.Linear(2,64) # N, 19, 2
        
        self.encoder = nn.LSTM(128, 128, 1) # input 19, N, 64 output 1, N, 64 

        self.decoder_p = nn.LSTM(128, 128, 1) # input 30, N, 64 output 30, N, 64
        self.decoder_v = nn.LSTM(128, 128,1) # input 30, N, 64 output 30, N, 64

        self.p_out = nn.Linear(128,2)
        self.v_out = nn.Linear(128,2)

    def forward(self, p, v):
        batch = p.shape[0]
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p,x_v),dim=2)
        x = x.permute(1,0,2)

        _,(state_h) = self.encoder(x)

        temp = torch.zeros(11,batch,128).to(device)
        x = torch.cat((x,temp),dim=0)

        x_p,_ = self.decoder_p(x,state_h)
        x_v,_ = self.decoder_v(x,state_h)

        x_p = x_p.permute(1,0,2)
        x_v = x_v.permute(1,0,2)

        x_p = self.p_out(x_p)
        x_v = self.v_out(x_v)

        # batch 50, 345s per epoch
        # p to p -> 9.21 after 10 epochs
        # p + v to p -> 8.8 after 10 epochs
        # p + v to p +v -> 6.65 after 10 epochs

        # batch 100, 345s per epoch
        # p + v to p +v -> 4.5 after 10 epochs

        return x_p , x_v

In [ ]:
'''
start with initial 19 points and zero fill remaining, bigger
'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2,64) # N, 19, 2
        self.v_in = nn.Linear(2,64) # N, 19, 2
        
        self.convert = nn.Linear(128,256)

        self.encoder = nn.LSTM(128,256,1)
        self.encoder_2 = nn.LSTM(256, 256, 1) # input 19, N, 256 output 1, N, 256

        self.decoder_p = nn.LSTM(256, 256, 1) 
        self.decoder_p_2 = nn.LSTM(256, 512, 1) 

        self.decoder_v = nn.LSTM(256, 256,1) 
        self.decoder_v_2 = nn.LSTM(256, 512,1) 

        self.p_out = nn.Linear(512,256)
        self.p_out_2 = nn.Linear(256,2)
        self.v_out = nn.Linear(512,256)
        self.v_out_2 = nn.Linear(256,2)


    def forward(self, p, v):
        batch = p.shape[0]
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p,x_v),dim=2)
        x = x.permute(1,0,2)

        x_temp,_ = self.encoder(x)
        _,(state_h) = self.encoder_2(x_temp)

        temp = torch.zeros(11,batch,128).to(device)

        x = torch.cat((x,temp),dim=0)

        x = self.convert(x)

        x_p,_ = self.decoder_p(x,state_h)
        x_p,_ = self.decoder_p_2(x_p)

        x_v,_ = self.decoder_v(x,state_h)
        x_v,_ = self.decoder_v_2(x_v)

        x_p = x_p.permute(1,0,2)
        x_v = x_v.permute(1,0,2)

        x_p = F.relu(self.p_out(x_p))
        x_p = self.p_out_2(x_p)

        x_v = F.relu(self.v_out(x_v))
        x_v = self.v_out_2(x_v)


        # batch 50, 345s per epoch
        # p to p -> 9.21 after 10 epochs
        # p + v to p -> 8.8 after 10 epochs
        # p + v to p +v -> 6.65 after 10 epochs

        # batch 100, 345s per epoch
        # p + v to p +v -> 4.5 after 10 epochs

        return x_p , x_v

In [ ]:
'''
Edward's Tests, bigger
'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2,32) # N, 19, 2
        self.v_in = nn.Linear(2,32) # N, 19, 2
        
        self.encoder = nn.LSTM(64, 128, 1) # input 19, N, 64 output 19, N, 128
        self.encoder_2 = nn.LSTM(128, 128, 1) # output 1, N, 128

        self.decoder_p = nn.LSTM(128, 256, 1) # input 30, N, 128 output 30, N, 256
        self.decoder_p_2 = nn.LSTM(256, 256, 1) # input 30, N, 256 output 30, N, 256

        self.decoder_v = nn.LSTM(128, 256,1) # input 30, N, 128 output 30, N, 256
        self.decoder_v_2 = nn.LSTM(256, 256,1) # input 30, N, 256 output 30, N, 256

        self.p_out = nn.Linear(256,128)
        self.p_out_2 = nn.Linear(128,2)
        self.v_out = nn.Linear(256,128)
        self.v_out_2 = nn.Linear(128,2)

    def forward(self, p, v):
        batch = p.shape[0]
        
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p,x_v),dim=2)
        x = x.permute(1,0,2)

        x, _ = self.encoder(x)
        _,(state_h,_) = self.encoder_2(x)

        x = state_h.repeat(30,1,1)     

        x_p,_ = self.decoder_p(x)
        x_p,_ = self.decoder_p_2(x_p)

        x_v,_ = self.decoder_v(x)
        x_v,_ = self.decoder_v_2(x_v)

        x_p = x_p.permute(1,0,2)
        x_v = x_v.permute(1,0,2)

        x_p = F.relu(self.p_out(x_p))
        x_p = self.p_out_2(x_p)

        x_v = F.relu(self.v_out(x_v))
        x_v = self.v_out_2(x_v)


        # batch 50, 345s per epoch
        # p to p -> 9.21 after 10 epochs
        # p + v to p -> 8.8 after 10 epochs
        # p + v to p +v -> 6.65 after 10 epochs

        # batch 100, 345s per epoch
        # p + v to p +v -> 4.5 after 10 epochs

        return x_p , x_v

In [ ]:
'''
Edward's Tests, bigger, bigger (not as good)
'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2,32) # N, 19, 2
        self.v_in = nn.Linear(2,32) # N, 19, 2
        
        self.encoder = nn.LSTM(64, 128, 1) # input 19, N, 64 output 19, N, 128
        self.encoder_2 = nn.LSTM(128, 128, 1) # output 1, N, 128

        self.translate = nn.Linear(128,256) # translation

        self.decoder_p = nn.LSTM(256, 512, 1) # input 30, N, 128 output 30, N, 256
        self.decoder_p_2 = nn.LSTM(512,512, 1) # input 30, N, 256 output 30, N, 256

        self.decoder_v = nn.LSTM(256, 512,1) # input 30, N, 128 output 30, N, 256
        self.decoder_v_2 = nn.LSTM(512, 512,1) # input 30, N, 256 output 30, N, 256

        self.p_out = nn.Linear(512,256)
        self.p_out_2 = nn.Linear(256,2)
        self.v_out = nn.Linear(512,256)
        self.v_out_2 = nn.Linear(256,2)

    def forward(self, p, v):
        batch = p.shape[0]
        
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p,x_v),dim=2)
        x = x.permute(1,0,2)

        x, _ = self.encoder(x)
        _,(state_h,_) = self.encoder_2(x)

        state_h = self.translate(state_h)

        x = state_h.repeat(30,1,1)     

        x_p,_ = self.decoder_p(x)
        x_p,_ = self.decoder_p_2(x_p)

        x_v,_ = self.decoder_v(x)
        x_v,_ = self.decoder_v_2(x_v)

        x_p = x_p.permute(1,0,2)
        x_v = x_v.permute(1,0,2)

        x_p = F.relu(self.p_out(x_p))
        x_p = self.p_out_2(x_p)

        x_v = F.relu(self.v_out(x_v))
        x_v = self.v_out_2(x_v)


        # batch 50, 345s per epoch
        # p to p -> 9.21 after 10 epochs
        # p + v to p -> 8.8 after 10 epochs
        # p + v to p +v -> 6.65 after 10 epochs

        # batch 100, 345s per epoch
        # p + v to p +v -> 4.5 after 10 epochs

        return x_p , x_v

In [ ]:
'''
original network
'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2,32) # N, 19, 2
        self.v_in = nn.Linear(2,32) # N, 19, 2
        
        self.encoder = nn.LSTM(64, 64, 1) # input 19, N, 64 output 1, N, 64 

        self.decoder_p = nn.LSTM(64, 128, 1) # input 30, N, 64 output 30, N, 128
        self.decoder_v = nn.LSTM(64, 128,1) # input 30, N, 64 output 30, N, 128

        self.p_out = nn.Linear(128,2)
        self.v_out = nn.Linear(128,2)

    def forward(self, p, v):
        batch = p.shape[0]
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p,x_v),dim=2)
        x = x.permute(1,0,2)

        _,(state_h,_) = self.encoder(x)

        x = state_h.repeat(30,1,1).to(device)     

        x_p,_ = self.decoder_p(x)
        x_v,_ = self.decoder_v(x)

        x_p = x_p.permute(1,0,2)
        x_v = x_v.permute(1,0,2)

        x_p = self.p_out(x_p)
        x_v = self.v_out(x_v)

        # batch 50, 345s per epoch
        # p to p -> 9.21 after 10 epochs
        # p + v to p -> 8.8 after 10 epochs
        # p + v to p +v -> 6.65 after 10 epochs

        # batch 100, 345s per epoch
        # p + v to p +v -> 4.5 after 10 epochs

        return x_p , x_v

In [ ]:
# don't run

import numpy as np
model = Trajectory().double().to(device)
print(model)

p = np.zeros((4,19,2))
v = np.zeros((4,19,2))
p_torch = torch.tensor(p).to(device)
v_torch = torch.tensor(v).to(device)

output = model(p_torch,v_torch)
print(len(output))
print(output[0].shape)

Trajectory(
  (p_in): Linear(in_features=2, out_features=64, bias=True)
  (v_in): Linear(in_features=2, out_features=64, bias=True)
  (convert): Linear(in_features=128, out_features=256, bias=True)
  (encoder): LSTM(128, 256)
  (encoder_2): LSTM(256, 256)
  (decoder_p): LSTM(256, 256)
  (decoder_p_2): LSTM(256, 512)
  (decoder_v): LSTM(256, 256)
  (decoder_v_2): LSTM(256, 512)
  (p_out): Linear(in_features=512, out_features=256, bias=True)
  (p_out_2): Linear(in_features=256, out_features=2, bias=True)
  (v_out): Linear(in_features=512, out_features=256, bias=True)
  (v_out_2): Linear(in_features=256, out_features=2, bias=True)
)
2
torch.Size([4, 30, 2])


In [ ]:
import time
from tqdm import tqdm

batch_sz = 100
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train, num_workers=2)

train_size = int(0.8 * len(train_loader.dataset))
val_size = len(train_loader.dataset) - train_size
train_data, val_data = torch.utils.data.random_split(train_loader.dataset, [train_size, val_size])

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data), "\nLENGTH OF VAL DATA:", len(val_data))

train_data = DataLoader(train_data, batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train, num_workers=2)
val_data = DataLoader(val_data, batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train, num_workers=2)
model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters(), lr=0.001)

# decayRate = 0.999    
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=my_optim, gamma=decayRate)

epoch = 20 # takes around 20 epochs to converge
number = 1 # number of cars in each 

for i in range(epoch):

  model.train()
  epoch_loss = 0
  start = time.time()
  
  train_loop = tqdm(enumerate(train_data), total=len(train_data))

  for i_batch, sample_batch in train_loop:
    inp, out = sample_batch
    batch = inp.shape[0]

    p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800. # -2400.)/2400.
    v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
    p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. # -2400.)/2400.
    v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.

    pred = model(p_in, v_in)


    loss = 0
    p_criteria = nn.MSELoss()
    p_loss = torch.sqrt(p_criteria(pred[0], p_out))

    v_criteria = nn.MSELoss()
    v_loss = torch.sqrt(v_criteria(pred[1], v_out))

    loss = p_loss + v_loss
    epoch_loss += p_loss

    my_optim.zero_grad()
    loss.backward()
    my_optim.step()

    train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
    train_loop.set_postfix(loss = epoch_loss.item())
    # my_lr_scheduler.step()
    # if(i>2999):
    #   # print(pred[0]*2400.+2400., p_out[0]*2400.+2400.)
    #   print(pred[0]*4800., p_out[0]*4800.)
    # break
  #print("Training Loss: ", i, epoch_loss.item(), time.time() - start)
  
  model.eval()
  epoch_loss = 0
  val_loop = tqdm(enumerate(val_data), total=len(val_data))
  for i_batch, sample_batch in val_loop:
    inp, out = sample_batch
    batch = inp.shape[0]

    p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800.
    v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
    p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. 
    v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.

    pred = model(p_in, v_in)

    loss = 0
    p_criteria = nn.MSELoss()
    p_loss = torch.sqrt(p_criteria(pred[0], p_out))

    epoch_loss += p_loss

    val_loop.set_description(f"Val.  Epoch [{i + 1}/{epoch}]")
    val_loop.set_postfix(loss = epoch_loss.item())
  #print("Validation Loss: ", epoch_loss.item())

In [ ]:
def perBatchMaxTor(array,size):
  temp = array[:,0]
  temp = torch.unsqueeze(temp,1)
  temp = torch.repeat_interleave(temp,size,1)
  return torch.max(array-temp)

def zeroNormNP(array, size=19, pos=np.array([-1,-1])): # array is N,19,2
  temp = []
  if( np.array_equal(pos, np.array([-1,-1]))):
    temp = array[:,0]
  else:
    temp = pos # should be N,2
  temp = np.expand_dims(temp,1)
  temp = np.repeat(temp,size,1)

  return (array-temp)/200., array[:,0]

def zeroNormTor(array, size=19, pos=torch.tensor([-1,-1])): # array is N,19,2
  temp = []
  if( np.array_equal(pos, torch.tensor([-1,-1]))):
    temp = array[:,0]
  else:
    temp = pos # should be N,2
  temp = torch.unsqueeze(temp,1)
  temp = torch.repeat_interleave(temp,size,1)
  return (array-temp)/200., array[:,0]

In [ ]:
# train on multiple cars per
import time

batch_sz = 100
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)

train_size = int(0.8 * len(train_loader.dataset))
val_size = len(train_loader.dataset) - train_size
train_data, val_data = torch.utils.data.random_split(train_loader.dataset, [train_size, val_size])

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data), "\nLENGTH OF VAL DATA:", len(val_data))

train_data = DataLoader(train_data, batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)
val_data = DataLoader(val_data, batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)
model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters(), lr=0.001)

# decayRate = 0.999    
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=my_optim, gamma=decayRate)

epoch = 30 # takes around 20 epochs to converge
number = 5 # number of cars in each 

best_val = 100

for i in range(epoch):

  model.train()
  epoch_loss = 0
  start = time.time()
  
  # train_loop = tqdm(enumerate(train_data), total=len(train_data))

  for i_batch, sample_batch in enumerate(train_data):

    inp, out = sample_batch
    batch = inp.shape[1]

    p_in, orig = zeroNormTor(inp[0].reshape(batch*number,19,2).to(device))
    v_in = inp[1].reshape(batch*number,19,2).to(device)/85.
    p_out,_ = zeroNormTor(out[0].reshape(batch*number,30,2).to(device),30, orig)
    v_out = out[1].reshape(batch*number,30,2).to(device)/85.

    pred = model(p_in, v_in)


    loss = 0
    p_criteria = nn.MSELoss()
    p_loss = torch.sqrt(p_criteria(pred[0], p_out))

    v_criteria = nn.MSELoss()
    v_loss = torch.sqrt(v_criteria(pred[1], v_out))

    loss = p_loss + v_loss
    epoch_loss += p_loss.item()

    my_optim.zero_grad()
    loss.backward()
    my_optim.step()

    # ur tqdm stuff broke
    # train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
    # train_loop.set_postfix(loss = epoch_loss.item()) 
    # my_lr_scheduler.step()
    # if(i>2999):
    #   temp = torch.unsqueeze(orig,1)
      # temp = torch.repeat_interleave(temp,30,1)
      # print((pred[0]*500+temp)[0], (p_out*500+temp)[0])
    # break
  print("Training Loss: ", i, epoch_loss, time.time() - start)
  
  model.eval()
  epoch_loss = 0
  # val_loop = tqdm(enumerate(val_data), total=len(val_data))
  for i_batch, sample_batch in enumerate(val_data):
    inp, out = sample_batch
    batch = inp.shape[1]

    p_in,orig = zeroNormTor(inp[0].reshape(batch*number,19,2).to(device))
    v_in = inp[1].reshape(batch*number,19,2).to(device)/85.
    p_out,_ = zeroNormTor(out[0].reshape(batch*number,30,2).to(device),30, orig)
    v_out = out[1].reshape(batch*number,30,2).to(device)/85.

    pred = model(p_in, v_in)

    loss = 0
    p_criteria = nn.MSELoss()
    p_loss = torch.sqrt(p_criteria(pred[0], p_out))

    epoch_loss += p_loss.item()

    # val_loop.set_description(f"Val.  Epoch [{i + 1}/{epoch}]")
    # val_loop.set_postfix(loss = epoch_loss.item())
  if(epoch_loss< best_val):
    filename = "best.pth"
    state = {'epoch': i, 'state_dict': model.state_dict(),
            'optimizer': my_optim.state_dict()}
    torch.save(state, filename)
    print("saved")
    best_val = epoch_loss

  print("Validation Loss: ", epoch_loss)


LENGTH OF TRAIN LOADER DATASET: 205942
LENGTH OF TRAIN DATA: 164753 
LENGTH OF VAL DATA: 41189
Training Loss:  0 28.060927054844797 296.6075711250305
saved
Validation Loss:  5.980197741650045
Training Loss:  1 20.87761565949768 307.6591205596924
saved
Validation Loss:  5.435493870638311
Training Loss:  2 19.3822606978938 309.7150640487671
Validation Loss:  5.555410088039935
Training Loss:  3 18.694232560694218 309.6612434387207
saved
Validation Loss:  4.736429109238088
Training Loss:  4 18.34214370744303 310.71124958992004
saved
Validation Loss:  4.531997323036194
Training Loss:  5 18.0427740868181 309.95675706863403
saved
Validation Loss:  4.456412710249424
Training Loss:  6 17.773441360797733 311.14957547187805
saved
Validation Loss:  4.381199871189892
Training Loss:  7 17.63086613547057 309.7604398727417
Validation Loss:  4.42516256403178
Training Loss:  8 17.39989645173773 310.13033151626587
Validation Loss:  4.475255382247269
Training Loss:  9 17.22623194148764 309.0701665878296


In [ ]:
import time 

batch_sz = 100
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train, num_workers=2)

model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters())

# decayRate = 0.999
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=my_optim, gamma=decayRate)

epoch = 20 # takes around 20 epochs to converge
number = 1 # number of cars in each 

for i in range(epoch):

  model.train()
  epoch_loss = 0
  start = time.time()
  for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    batch = inp.shape[0]

    p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800. # -2400.)/2400.
    v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
    p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. # -2400.)/2400.
    v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.

    pred = model(p_in, v_in)


    loss = 0
    p_criteria = nn.MSELoss()
    p_loss = torch.sqrt(p_criteria(pred[0], p_out))

    v_criteria = nn.MSELoss()
    v_loss = torch.sqrt(v_criteria(pred[1], v_out))

    loss = p_loss + v_loss
    epoch_loss += p_loss

    my_optim.zero_grad()
    loss.backward()
    my_optim.step()
    # my_lr_scheduler.step()
    # if(i>2999):
    #   # print(pred[0]*2400.+2400., p_out[0]*2400.+2400.)
    #   print(pred[0]*4800., p_out[0]*4800.)
    # break
  print("Training Loss: ", i, epoch_loss.item(), time.time() - start)
  
  model.eval()
  epoch_loss = 0
  


In [ ]:
model.eval()
load = True 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
initial = 0

model.eval()

if(load):
  checkpoint = torch.load('best.pth')
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  print("=> loaded checkpoint '{}' (epoch {})".format(filename, checkpoint['epoch']))
  initial += int(checkpoint['epoch'])

batch_sz = 100
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)

epoch_loss =0
batch = 60
number = 5

for i_batch, sample_batch in enumerate(train_loader):
  inp, out = sample_batch
  batch = inp.shape[1]

  p_in = (inp[0].reshape(batch*number,19,2).to(device))/4800.
  v_in = (inp[1].reshape(batch*number,19,2).to(device)+85.)/170.
  p_out = (out[0].reshape(batch*number,30,2).to(device))/4800.
  v_out = (out[1].reshape(batch*number,30,2).to(device)+85.)/170.

  pred = model(p_in, v_in)
  loss = 0
  p_criteria = nn.MSELoss()
  p_loss = torch.sqrt(p_criteria(pred[0], p_out))

  epoch_loss += p_loss.item()
  print(pred[0][10]*4800., p_out[10]*4800.)
  break
  
print("Validation Loss: ", epoch_loss)

In [ ]:
import csv 
load = True 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
initial = 0

model.eval()

if(load):
  checkpoint = torch.load('best.pth')
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  print("=> loaded checkpoint '{}' (epoch {})".format(filename, checkpoint['epoch']))
  initial += int(checkpoint['epoch'])


temp = []

new_path = "/content/new_val_in"
val_dataset  = ArgoverseDataset(data_path=new_path)

top = []
top.append("ID")
for i in range(60):
  top.append("v"+str(i+1))
temp.append(top)

batch = 60
number = 1

with torch.no_grad():
    for i in val_dataset:
      row = []
      scene = i['scene_idx']
      agent = i['agent_id']
      target =0
      for x in range(len(i['track_id'])):
        if i['track_id'][x][0] == agent:
          target = x

      p_in = torch.LongTensor(i['p_in'])
      v_in = torch.LongTensor(i['v_in'])

      p_in = (p_in.reshape(batch*number,19,2).to(device))/4800.
      v_in = (v_in.reshape(batch*number,19,2).to(device)+85.)/170.

      pred = model(p_in, v_in)

      pred_out = pred[0]*4800.
      
      output = pred_out[target]

      row.append(scene)
      row = row + torch.flatten(output).cpu().detach().numpy().tolist()
      temp.append(row)

with open('submission4.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(temp)

=> loaded checkpoint 'best.pth' (epoch 18)
